In [34]:
#open modified sentiment file 
def load_modified_senti_file():
    with open("ENdev.final.out", encoding="utf-8") as file:
        modified_train_list = file.readlines()
        modified_train_list = [x.strip() for x in modified_train_list]
        return modified_train_list
        
modifiedEN_train_list = load_modified_senti_file()

annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}

for i in range(len(modifiedEN_train_list)):
    annotated_word = modifiedEN_train_list[i]
    each_word = annotated_word.split(" ")
    if (len(each_word) == 2):
        if each_word[1] == "O":
            annotationDict["O"] += 1
        elif each_word[1] == "I-positive":
            annotationDict["I-positive"] += 1
        elif each_word[1] == "B-positive":
            annotationDict["B-positive"] += 1
        elif each_word[1] == "I-negative":
            annotationDict["I-negative"] += 1
        elif each_word[1] == "B-negative":
            annotationDict["B-negative"] += 1
        elif each_word[1] == "I-neutral":
            annotationDict["I-neutral"] += 1
        elif each_word[1] == "B-neutral":
            annotationDict["B-neutral"] += 1

print(annotationDict)
def separate_sentitweets():
    countStart = 1
    countBottom = 0
    countTop = 0
    # counting starts
    giantsenti_each_tweet = []
    each_tweet = []
    for i in range(len(modifiedEN_train_list)):
        annotated_word = modifiedEN_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word == ['']:
            countStart += 1
            giantsenti_each_tweet.append(each_tweet)
            each_tweet = []

        else:
            each_tweet.append(each_word[1])
    for i in range(len(giantsenti_each_tweet)):
        giantsenti_each_tweet[i].insert(0, 'START')
        giantsenti_each_tweet[i].append('STOP')
    return giantsenti_each_tweet, countStart
giantsenti_each_tweet, countStart = separate_sentitweets()

def estimate_transition(current_tag, previous_tag):
#     with open("modified_train_final2", encoding="utf-8") as file:
    startDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}
    for i in range(len(giantsenti_each_tweet)):
        each_tweet = giantsenti_each_tweet[i]
        if each_tweet[1] == "O":
            startDict["O"] += 1
        elif each_tweet[1] == "I-positive":
            startDict["I-positive"] += 1
        elif each_tweet[1] == "B-positive":
            startDict["B-positive"] += 1
        elif each_tweet[1] == "I-negative":
            startDict["I-negative"] += 1
        elif each_tweet[1] == "B-negative":
            startDict["B-negative"] += 1
        elif each_tweet[1] == "I-neutral":
            startDict["I-neutral"] += 1
        elif each_tweet[1] == "B-neutral":
            startDict["B-neutral"] += 1

    stopDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}
    for i in range(len(giantsenti_each_tweet)):
        each_tweet = giantsenti_each_tweet[i]
        last_index = len(each_tweet) - 2
        if each_tweet[last_index] == "O":
            stopDict["O"] += 1
        elif each_tweet[last_index] == "I-positive":
            stopDict["I-positive"] += 1
        elif each_tweet[last_index] == "B-positive":
            stopDict["B-positive"] += 1
        elif each_tweet[last_index] == "I-negative":
            stopDict["I-negative"] += 1
        elif each_tweet[last_index] == "B-negative":
            stopDict["B-negative"] += 1
        elif each_tweet[last_index] == "I-neutral":
            stopDict["I-neutral"] += 1
        elif each_tweet[last_index] == "B-neutral":
            stopDict["B-neutral"] += 1

    if previous_tag == "START":
        # current tag is Yn
        countTop = startDict[current_tag]
#             print(countTop)
#             print(countStart)
#             print(countTop/countStart)
        return countTop/countStart

    elif current_tag == "STOP":
        # previous tag is Yn
        countTop = stopDict[previous_tag]
        countBottom = annotationDict[previous_tag]
        return countTop/countBottom

    else:
        giantTransitionDict = {}
        for i in range(len(giantsenti_each_tweet)):
            each_tweet = giantsenti_each_tweet[i]
            for j in range(1,len(each_tweet)-1):
                if j == len(each_tweet):
                    break
                if each_tweet[j-1] == previous_tag and each_tweet[j] == "O":
                    if (previous_tag + "O") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "O"] += 1
                    else:
                        giantTransitionDict[previous_tag + "O"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] == "B-negative":
                    if (previous_tag + "B-negative") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "B-negative"] += 1
                    else:
                        giantTransitionDict[previous_tag + "B-negative"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] == "B-positive":
                    if (previous_tag + "B-positive") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "B-positive"] += 1
                    else:
                        giantTransitionDict[previous_tag + "B-positive"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] =="B-neutral":
                    if (previous_tag + "B-neutral") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "B-neutral"] += 1
                    else:
                        giantTransitionDict[previous_tag + "B-neutral"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] =="I-positive":
                    if (previous_tag + "I-positive") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "I-positive"] += 1
                    else:
                        giantTransitionDict[previous_tag + "I-positive"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] =="I-negative":
                    if (previous_tag + "I-negative") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "I-negative"] += 1
                    else:
                        giantTransitionDict[previous_tag + "I-negative"] = 1

                elif each_tweet[j] == previous_tag and each_tweet[j+1] =="I-neutral":
                    if (previous_tag + "I-neutral") in giantTransitionDict:
                        giantTransitionDict[previous_tag + "I-neutral"] += 1
                    else:
                        giantTransitionDict[previous_tag + "I-neutral"] = 1
#             print(giantTransitionDict)
        if giantTransitionDict.get(previous_tag+current_tag) == None:
            return 0
        else:
            countTop = giantTransitionDict[previous_tag+current_tag]
            countBottom = annotationDict[previous_tag]

            return countTop/countBottom

    return 1

estimate_transition("O", "START") #current, previous

#estimating emission from dev.out
def get_word_dict():
    modifiedWordDict = {}        
    for i in range(len(modifiedEN_train_list)):
        annotated_word = modifiedEN_train_list[i]
        each_word = annotated_word.split(" ")
        if each_word[0] in modifiedWordDict:
            modifiedWordDict[each_word[0]] += 1
        else:
            modifiedWordDict[each_word[0]] = 1

    return modifiedWordDict
    
modifiedWordDict = get_word_dict()
def estimate_emission_fix(word, tag):
        countTop = 0
        countBottom = annotationDict[tag]
        if word in modifiedWordDict:
            # do nothing
            word = word
            
        else:
            word = "#UNK#"
            
        for i in range(len(modifiedEN_train_list)):
            annotated_word = modifiedEN_train_list[i]
            each_word = annotated_word.split(" ")
            if (each_word[0] == word and each_word[1] == tag):
                countTop += 1
#         print('emission')
#         print(countTop)
        return countTop/countBottom

estimate_emission_fix("I", "O")


    
    

{'O': 2667, 'I-positive': 750, 'B-positive': 80, 'I-negative': 115, 'B-negative': 107, 'I-neutral': 203, 'B-neutral': 124}


0.0307461567304087

In [35]:
#store transition end emission probabilities
probabilitydict={}
start=0
for i in range(len(modifiedEN_train_list)):
    k=modifiedEN_train_list[i]
    if k=="":
        start=0
    else:
        k=k.split(" ")
        word=k[0]
        tag=k[1]
        key=k[0]+","+k[1]
        #print(i[1])
        #calulate transition at start
        if start==0:
            previoustag="START"
            if key in probabilitydict:
                pass
            else:
                probabilitydict[key]=estimate_emission_fix(word,tag)
                start=1
        transition=previoustag+","+tag
        if modifiedEN_train_list[i+1]=="":
            transit=tag+","+"STOP"
            if transit in probabilitydict:
                pass
            else:
                probabilitydict[transit]=estimate_transition("STOP",tag)
        elif transition in probabilitydict:
            pass
        else:
            probabilitydict[transition]=estimate_transition(tag,previoustag)
        previoustag=k[1]
    #do for stop case
        
    
print(probabilitydict)

{'When,O': 0.0007499062617172854, 'START,O': 0.6591760299625468, 'O,O': 0.6212973378327709, 'O,I-positive': 0.07424071991001124, 'I-positive,O': 0.5973333333333334, 'O,I-negative': 0.004124484439445069, 'I-negative,O': 0.5304347826086957, 'I-positive,I-positive': 0.19066666666666668, 'I-positive,B-positive': 0.024, 'B-positive,O': 0.6625, 'O,STOP': 0.09673790776152981, 'The,B-neutral': 0.3709677419354839, 'START,B-neutral': 0.20224719101123595, 'B-neutral,B-negative': 0.0967741935483871, 'B-negative,I-neutral': 0.12149532710280374, 'I-neutral,I-positive': 0.18226600985221675, 'My,O': 0.003374578177727784, 'I-positive,I-neutral': 0.09066666666666667, 'I-neutral,B-positive': 0.019704433497536946, 'B-positive,I-negative': 0.025, 'Food,B-neutral': 0.016129032258064516, 'B-neutral,I-positive': 0.1693548387096774, 'O,B-negative': 0.006749156355455568, 'B-negative,I-positive': 0.12149532710280374, 'I-positive,B-negative': 0.02266666666666667, '–,O': 0.0029996250468691415, 'I-positive,I-negati

In [ ]:
def forward_prob(i,u):
    # base case
    if i == 1:
        a = transition from start to u
        return a
    # recursive case
    else:
        forward_prob_list = []
        for v in all_tags:
            if probabilitydict.get(v+","+u) != None and probabilitydict.get(v+","+word at u) != None:
                a = probabilitydict[v+","+u]#transition from v to u #read from dict
                b = probabilitydict[v+","+word at u]#emission from v to xi #read from dict
                forward_prob_from_v = forward_prob(i-1, v)*a*b
                forward_prob_list.append(forward_prob_from_v)
        return sum(forward_prob_list)
    
def back_prob(i,u, lengthOfSentence):
    n = lengthOfSentence
    # base case
    if i == n:
        a = transition from u to stop
        b = emission from u to xn
        return a*b
    # recursive case
    else:
        back_prob_list = []
        for v in all_tags:
            if probabilitydict.get(u+","+v) != None and probabilitydict.get(u+","+word at u) != None:
                a = transition from u to v
                b = emission from u to xi
                back_prob_from_v = back_prob(i + 1, v, n) * a * b
                back_prob_list.append(forward_prob_from_v)
        return sum(back_prob_list)


In [ ]:
def max_marginal(sentence):
    #sentence is a list containing the words
    # this is only for one sentence
    opti_path = []
    for i in range(1, len(sentence) + 1):
        opti_y_dict = {}
        for tag in all_tags:
            opti_y_at_i = forward_prob(i,tag)*back_prob(i,tag, len(sentence))
            opti_y_dict[tag] = opti_y_at_i
        max_i_path = argmax(opti_y_dict)
        opti_path.append(max_i_path)
        
    return opti_path

# dev_in_list is a giant list
# do a split according to ""
for sentence in sentences:
    max_marginal(sentence)

In [2]:
def factorial(n):

    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

print(factorial(3))

3
2
1
6
